In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('ASRS_10y_data.csv')

/Users/hicheme/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (15,19,40,41,48,67,71,72,78,79,81,89,99) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# Set pandas settings to show all data when using .head(), .columns etc.
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [29]:
# Taking a first look at the shape of the dataframe
df.shape

# the dataframe is constiituted with 24063 entries and # columns

(24063, 21)

In [4]:
#taking a look at the information of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24063 entries, 0 to 24062
Columns: 126 entries, ACN to Unnamed: 125
dtypes: float64(29), int64(3), object(94)
memory usage: 23.1+ MB


In [8]:
# Exploratory data analysis
df.isnull().sum()

ACN                                                                  0
Date                                                                 0
Local Time Of Day                                                 1224
Locale Reference                                                     3
State Reference                                                      0
Relative Position.Angle.Radial                                   22928
Relative Position.Distance.Nautical Miles                        21842
Altitude.AGL.Single Value                                        16904
Altitude.MSL.Single Value                                        10844
Latitude / Longitude (UAS)                                       24063
Flight Conditions                                                 8996
Weather Elements / Visibility                                    17880
Work Environment Factor                                          23926
Light                                                             8877
Ceilin

In [25]:
# The first impression that we can get while looking to all the data is that there is quite large number of
# features where you'd find a lot of missing value. I'D suggest to start by removing those features as we can not
# replace the data when 90% of the data are missing

#I'll procede by droping the columns where you'd find more than 90% of missing data

null_value =[]
for col in df:
    num = df[col].isnull().sum()
    if num>20000:
        df = df.drop(col, axis =1)

In [30]:
df.info()
# the Dtype of the majjority of the columns is object
# I procede by selecting the data of the dataframe where the Dtype is an object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24063 entries, 0 to 24062
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   ACN                                  24063 non-null  int64 
 1   Date                                 24063 non-null  int64 
 2   Local Time Of Day                    22839 non-null  object
 3   Locale Reference                     24060 non-null  object
 4   State Reference                      24063 non-null  object
 5   Make Model Name                      23989 non-null  object
 6   Operating Under FAR Part             22185 non-null  object
 7   Flight Phase                         23485 non-null  object
 8   Location Of Person                   23842 non-null  object
 9   Reporter Organization                23558 non-null  object
 10  Function                             23908 non-null  object
 11  Qualification                        2206

In [46]:
# One of the features that sparked my interest is the Flight Phase, let's group all of the DataFrame based on 
# this feature and see at what phases we would get the most of our information
df.groupby(df_object['Flight Phase']).count()


,ACN,Date,Local Time Of Day,Locale Reference,State Reference,Make Model Name,Operating Under FAR Part,Flight Phase,Location Of Person,Reporter Organization,Function,Qualification,ASRS Report Number.Accession Number,Anomaly,Detector,When Detected,Result,Contributing Factors / Situations,Primary Problem,Narrative,Synopsis
Flight Phase,,,,,,,,,,,,,,,,,,,,,
Climb,2290,2290,2200,2290,2290,2289,2147,2290,2272,2258,2282,2107,2290,2289,2265,2140,2194,2280,2280,2290,2290
Climb; Cruise,19,19,18,19,19,19,16,19,19,18,19,18,19,19,18,19,16,19,19,19,19
Climb; Cruise; Descent,18,18,16,18,18,18,10,18,18,18,17,18,18,18,18,16,17,18,18,18,18
Climb; Cruise; Descent; Initial Climb,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Climb; Cruise; Initial Climb,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2
Climb; Cruise; Parked; Landing; Descent,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Climb; Descent,5,5,5,5,5,5,5,5,5,5,5,4,5,5,5,5,4,5,5,5,5
Climb; Descent; Cruise,10,10,10,10,10,10,6,10,10,10,10,9,10,10,10,8,10,10,10,10,10
Climb; Final Approach,5,5,5,5,5,5,5,5,5,4,5,5,5,5,5,4,5,5,5,5,5


In [43]:
# Interesting, you can see that our features are not equally distributes according to the flight phase
# the most important part of the data can be found either in: 
   # the climb phase
   # the cruise phase
   # the descent phase
   # the final approach phase
   # the initial approach phase
   # the initial climb (isn't that the same with the climb phase, is yes we can replace the name)
   # the landing phase
   # Parked phas

,ACN,Date,Local Time Of Day,Locale Reference,State Reference,Make Model Name,Operating Under FAR Part,Flight Phase,Location Of Person,Reporter Organization,Function,Qualification,ASRS Report Number.Accession Number,Anomaly,Detector,When Detected,Result,Contributing Factors / Situations,Primary Problem,Narrative,Synopsis
0,986560,201201,1801-2400,A11.TRACON,AK,Super King Air 200,Part 135,Descent,Facility A11.TRACON,Government,Departure; Approach,Air Traffic Control Developmental,986560,Deviation - Altitude Excursion From Assigned A...,Person Air Traffic Control,NaN,Air Traffic Control Issued New Clearance,Human Factors,Human Factors,A BE20 was inbound to ANC via YESKA. He was to...,A11 Controller described a descent below MVA; ...
1,988077,201201,0001-0600,ENA.Airport,AK,Caravan Undifferentiated,Part 135,Final Approach,Facility ZAN.ARTCC,Government,Enroute,Air Traffic Control Fully Certified,988077,ATC Issue All Types; Deviation / Discrepancy -...,Person Air Traffic Control,NaN,General None Reported / Taken,Weather; Human Factors; Procedure,Human Factors,A C208 departed for Kenai. Kenai Tower called ...,ZAN Controller issued a Visual Approach cleara...
2,988094,201201,0601-1200,PHX.Airport,AZ,B737 Next Generation Undifferentiated,Part 121,Descent,Aircraft X,Air Carrier,Pilot Flying; First Officer,NaN,988094,Deviation - Altitude Crossing Restriction Not ...,Person Flight Crew,In-flight,Flight Crew Returned To Clearance; Flight Crew...,Procedure; Aircraft; Human Factors,Aircraft,We were on the GEELA Arrival into PHX and were...,A B737-NG on the PHX GEELA arrival was vectore...
3,988431,201201,1801-2400,SDL.Airport,AZ,Viking,Part 91,Taxi,Facility SDL.Tower,Government,Ground,Air Traffic Control Fully Certified,988431,Conflict Ground Conflict; Critical; Deviation ...,Person Air Traffic Control,Taxi,General None Reported / Taken,Airport; Human Factors,Airport,While working Ground Control Aircraft X exited...,SDL Controller described a near collision with...
4,989412,201201,0601-1200,LIT.Airport,AR,PA-28 Cherokee/Archer/Dakota/Pillan/Warrior,Part 91,Cruise,Aircraft X,Personal,Single Pilot,Flight Crew Private,989412,Inflight Event / Encounter CFTT / CFIT; Inflig...,Automation Aircraft Terrain Warning; Person Fl...,In-flight,Air Traffic Control Provided Assistance; Fligh...,Human Factors; Weather,Human Factors,My brother; who is not a pilot; was my only pa...,PA28 pilot reports departing on a local pleasu...
5,989663,201201,1201-1800,P50.TRACON,AZ,Citation Excel (C560XL),Part 91,Cruise,Facility P50.TRACON,Government,Approach; Departure,Air Traffic Control Fully Certified,989663,Conflict Airborne Conflict; Deviation - Altitu...,Person Air Traffic Control,In-flight,General None Reported / Taken,Aircraft; Airspace Structure; Procedure,Procedure,A C560 was vectored northbound over PHX at [50...,P50 Controller described a TCAS RA event resul...
6,989967,201201,0001-0600,FFZ.Airport,AZ,Amateur/Home Built/Experimental,Part 91,Initial Approach,Aircraft X,Personal,Single Pilot,Flight Crew Commercial; Flight Crew Instrument...,989967,Conflict NMAC,Person Flight Crew,In-flight,Air Traffic Control Issued New Clearance,Environment - Non Weather Related; Human Facto...,Ambiguous,I was cleared into the Class D by Falcon Tower...,Light aircraft pilot reported an NMAC with ano...
7,989983,201201,0601-1200,CGZ.Airport,AZ,PA-28 Cherokee/Archer/Dakota/Pillan/Warrior,Part 91,Initial Approach,Aircraft X,FBO,Pilot Not Flying; Instructor,Flight Crew Multiengine; Flight Crew Instrumen...,989983,Conflict NMAC; Deviation / Discrepancy - Proce...,Person Flight Crew,In-flight,Flight Crew Took Evasive Action,Procedure; Human Factors,Human Factors,While practicing instrument approaches at Casa...,An Instructor pilot took evasive action from a...
8,990012,201201,0601-1200,TUS.Airport,AZ,Medium Large Transport; Low Wing; 2 Turbojet Eng,Part 121,Initial Approach,Aircraft X,Air Carrier,Captain; Pilot Not Flying,Flight Crew Air Transport Pilot (ATP),990012,Inflight Event 

In [62]:
# A closer look to the data made me more interested about 2 features, 
 # ACN column
 # ASRS Report Number column 

# Next, I will look if all the data are similar in the table
# I first look at the count of the 2 variable
print(df['ACN'].count())
print(df['ASRS Report Number.Accession Number'].count())

24063
24063


In [65]:
# We have "24063" values in both columns
# I know search for the number entries where the two variable are similar
i =0
for num in range(24063): 
    if df['ACN'].iloc[num]== df['ASRS Report Number.Accession Number'].iloc[num]:
        i =i+1

print(i)

24062


In [71]:
# the two columns have similar values in each entries
# these informations allow us to say the two columns give similar informations. We can drop one of them
# I will drop the df['ASRS Report Number.Accession Number'] column

df = df.drop('ASRS Report Number.Accession Number', axis =1)
df.head(20)

,ACN,Date,Local Time Of Day,Locale Reference,State Reference,Make Model Name,Operating Under FAR Part,Flight Phase,Location Of Person,Reporter Organization,Function,Qualification,Anomaly,Detector,When Detected,Result,Contributing Factors / Situations,Primary Problem,Narrative,Synopsis
0,986560,201201,1801-2400,A11.TRACON,AK,Super King Air 200,Part 135,Descent,Facility A11.TRACON,Government,Departure; Approach,Air Traffic Control Developmental,Deviation - Altitude Excursion From Assigned A...,Person Air Traffic Control,NaN,Air Traffic Control Issued New Clearance,Human Factors,Human Factors,A BE20 was inbound to ANC via YESKA. He was to...,A11 Controller described a descent below MVA; ...
1,988077,201201,0001-0600,ENA.Airport,AK,Caravan Undifferentiated,Part 135,Final Approach,Facility ZAN.ARTCC,Government,Enroute,Air Traffic Control Fully Certified,ATC Issue All Types; Deviation / Discrepancy -...,Person Air Traffic Control,NaN,General None Reported / Taken,Weather; Human Factors; Procedure,Human Factors,A C208 departed for Kenai. Kenai Tower called ...,ZAN Controller issued a Visual Approach cleara...
2,988094,201201,0601-1200,PHX.Airport,AZ,B737 Next Generation Undifferentiated,Part 121,Descent,Aircraft X,Air Carrier,Pilot Flying; First Officer,NaN,Deviation - Altitude Crossing Restriction Not ...,Person Flight Crew,In-flight,Flight Crew Returned To Clearance; Flight Crew...,Procedure; Aircraft; Human Factors,Aircraft,We were on the GEELA Arrival into PHX and were...,A B737-NG on the PHX GEELA arrival was vectore...
3,988431,201201,1801-2400,SDL.Airport,AZ,Viking,Part 91,Taxi,Facility SDL.Tower,Government,Ground,Air Traffic Control Fully Certified,Conflict Ground Conflict; Critical; Deviation ...,Person Air Traffic Control,Taxi,General None Reported / Taken,Airport; Human Factors,Airport,While working Ground Control Aircraft X exited...,SDL Controller described a near collision with...
4,989412,201201,0601-1200,LIT.Airport,AR,PA-28 Cherokee/Archer/Dakota/Pillan/Warrior,Part 91,Cruise,Aircraft X,Personal,Single Pilot,Flight Crew Private,Inflight Event / Encounter CFTT / CFIT; Inflig...,Automation Aircraft Terrain Warning; Person Fl...,In-flight,Air Traffic Control Provided Assistance; Fligh...,Human Factors; Weather,Human Factors,My brother; who is not a pilot; was my only pa...,PA28 pilot reports departing on a local pleasu...
5,989663,201201,1201-1800,P50.TRACON,AZ,Citation Excel (C560XL),Part 91,Cruise,Facility P50.TRACON,Government,Approach; Departure,Air Traffic Control Fully Certified,Conflict Airborne Conflict; Deviation - Altitu...,Person Air Traffic Control,In-flight,General None Reported / Taken,Aircraft; Airspace Structure; Procedure,Procedure,A C560 was vectored northbound over PHX at [50...,P50 Controller described a TCAS RA event resul...
6,989967,201201,0001-0600,FFZ.Airport,AZ,Amateur/Home Built/Experimental,Part 91,Initial Approach,Aircraft X,Personal,Single Pilot,Flight Crew Commercial; Flight Crew Instrument...,Conflict NMAC,Person Flight Crew,In-flight,Air Traffic Control Issued New Clearance,Environment - Non Weather Related; Human Facto...,Ambiguous,I was cleared into the Class D by Falcon Tower...,Light aircraft pilot reported an NMAC with ano...
7,989983,201201,0601-1200,CGZ.Airport,AZ,PA-28 Cherokee/Archer/Dakota/Pillan/Warrior,Part 91,Initial Approach,Aircraft X,FBO,Pilot Not Flying; Instructor,Flight Crew Multiengine; Flight Crew Instrumen...,Conflict NMAC; Deviation / Discrepancy - Proce...,Person Flight Crew,In-flight,Flight Crew Took Evasive Action,Procedure; Human Factors,Human Factors,While practicing instrument approaches at Casa...,An Instructor pilot took evasive action from a...
8,990012,201201,0601-1200,TUS.Airport,AZ,Medium Large Transport; Low Wing; 2 Turbojet Eng,Part 121,Initial Approach,Aircraft X,Air Carrier,Captain; Pilot Not Flying,Flight Crew Air Transport Pilot (ATP),Inflight Event / Encounter Other / Unknown,Automation Aircraft Terrain Warning; Person Fl...,In-flight,General Non

In [67]:
# Let me know look for another column that intreagued me, 
# It is the column 'When Detected'
# I'll procede with a value counts to see when the majority of the problems have been detected

df['When Detected'].value_counts()

In-flight                                                      17914
Taxi                                                            2499
Aircraft In Service At Gate                                      473
Pre-flight                                                       442
Routine Inspection                                               147
In-flight; Taxi                                                  109
Other takeoff roll                                                47
Taxi; In-flight                                                   42
Other Post Flight                                                 25
Other Takeoff roll                                                24
In-flight; Pre-flight                                             24
Aircraft In Service At Gate; In-flight                            23
Other Takeoff Roll                                                21
Other landing roll                                                21
Other Landing                     

In [68]:
# Interesting to see that the major part of the problems are detected :
  # In-flight
  # Taxi (really what does that mean lol?)